In [1]:
import pandas as pd
import plotly
import plotly.express as px 
import dtale

In [2]:
pd.set_option('display.max_rows', 20)  # Disable row limit
pd.set_option('display.max_columns', 40)  # Disable column limit
pd.set_option('display.width', 20)  # Disable line width limit
pd.set_option('display.max_colwidth', 100)  # Disable column width limit
# Set the display precision to 2 decimal places  
pd.set_option('display.precision', 2)  
  
# Or set the float format to always show 2 decimal places  
pd.set_option('display.float_format', '{:.2f}'.format)  


In [3]:
pd.options.plotting.backend = "plotly"
plotly.io.renderers.default = "notebook_connected"


In [4]:
df = pd.read_json("output/20241212_183425_real-estate-to-rent_real-estate-for-sale.jsonl", lines=True)
df.iloc[0].T

url                   https://www.bazaraki.com/adv/5415213_4-bedroom-detached-house-to-rent/
title                                                       4-bedroom detached house to rent
price                                                                                   1650
original_price                                                                           NaN
price_per_sqm                                                                            NaN
                                                       ...                                  
Density                                                                                  NaN
Coverage                                                                                 NaN
Registration block                                                                       NaN
Area                                                                                     NaN
Pick a point                                                          

In [6]:
def add_city_disctrict_cols(df):
    df[["city", "district"]] = df.location.str.split(",", expand=True)
    return df
df = add_city_disctrict_cols(df)
df.iloc[0].T

url                   https://www.bazaraki.com/adv/5415213_4-bedroom-detached-house-to-rent/
title                                                       4-bedroom detached house to rent
price                                                                                   1650
original_price                                                                           NaN
price_per_sqm                                                                            NaN
                                                       ...                                  
Registration block                                                                       NaN
Area                                                                                     NaN
Pick a point                                                                             NaN
city                                                                                 Larnaca
district                                                              

In [7]:
df.price_per_sqm = df.price / (df["Property area"]).round(2)

In [31]:
saledf = df[df.cat1.isin(['Houses for sale', 'Apartments, flats for sale'])]
rentdf = df[df.cat1.isin(['Houses to rent', 'Apartments, flats to rent'])]


In [ ]:
# selected_df = saledf[saledf.price < 5000]
# print(f"removed {len(saledf) - len(selected_df)} rows")
# df = selected_df

removed 12671 rows


In [ ]:
# selected_df = df.query("`Bedrooms` in ('3', '2', '1', 'Studio')")
# print(f"removed {len(df) - len(selected_df)} rows")
# df = selected_df

removed 0 rows


In [37]:
def stats(saledf, rentdf):
    salestatdf = saledf.query("Condition == 'Resale'").groupby(["city", "Bedrooms"]).price_per_sqm.describe()
    rentstatdf = rentdf.groupby(["city", "Bedrooms"]).price_per_sqm.describe()
    common_index = salestatdf.index.intersection(rentstatdf.index)
    result = rentstatdf.loc[common_index] / salestatdf.loc[common_index] * 12 * 100
    return pd.concat([result, rentstatdf, salestatdf], axis=1, keys=['result', 'rent', 'sale'])
    
print("apparts")
display(stats(saledf.query("cat1 == 'Apartments, flats for sale'"), rentdf.query("cat1 == 'Apartments, flats to rent'")))
print("houses")
display(stats(saledf.query("cat1 == 'Houses for sale'"), rentdf.query("cat1 == 'Houses to rent'")))

apparts


/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:4620: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:4620: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in s

result  \
                        count   
city      Bedrooms              
Famagusta 1            446.51   
          2            407.55   
          3            200.00   
          Studio      1200.00   
Larnaca   1           1555.56   
          2           1621.52   
          3            657.87   
          4           1090.91   
          Studio      1200.00   
Limassol  1           4050.00   
          2           2048.78   
          3           1687.50   
          4           1159.55   
          5            720.00   
          6 and more  1200.00   
          Studio     12000.00   
Nicosia   1           6316.98   
          2           2734.35   
          3           1981.74   
          4           2123.08   
          5            600.00   
          Studio      5657.14   
Paphos    1           1621.05   
          2           1663.29   
          3           1701.82   
          4            600.00   
          Studio      2133.33   
Famagusta 4               NaN   
          5               NaN   
Larnaca   6 and more      NaN   
Nicosia   6 and more      NaN   
Paphos    6 and more      NaN   

                            \
                      mean   
city      Bedrooms           
Famagusta 1           0.00   
          2           0.00   
          3           0.00   
          Studio      1.51   
Larnaca   1            inf   
          2            inf   
          3           0.00   
          4           7.44   
          Studio       inf   
Limassol  1            inf   
          2            NaN   
          3            NaN   
          4           3.80   
          5           3.99   
          6 and more 29.66   
          Studio      6.77   
Nicosia   1           7.29   
          2            NaN   
          3           5.81   
          4           5.03   
          5           1.71   
          Studio      8.57   
Paphos    1           5.92   
          2            NaN   
          3          44.80   
          4           5.18   
          Studio      9.77   
Famagusta 4            NaN   
          5            NaN   
Larnaca   6 and more   NaN   
Nicosia   6 and more   NaN   
Paphos    6 and more   NaN   

                             \
                        std   
city      Bedrooms            
Famagusta 1             NaN   
          2             NaN   
          3             NaN   
          Studio       0.09   
Larnaca   1             NaN   
          2             NaN   
          3             NaN   
          4           11.14   
          Studio        NaN   
Limassol  1             NaN   
          2             NaN   
          3             NaN   
          4            3.87   
          5            1.76   
          6 and more  92.36   
          Studio      15.82   
Nicosia   1           80.99   
          2             NaN   
          3            5.31   
          4            3.67   
          5             NaN   
          Studio       9.84   
Paphos    1            6.09   
          2             NaN   
          3          381.15   
          4             NaN   
          Studio      13.75   
Famagusta 4             NaN   
          5             NaN   
Larnaca   6 and more    NaN   
Nicosia   6 and more    NaN   
Paphos    6 and more    NaN   

                              \
                         min   
city      Bedrooms             
Famagusta 1             6.19   
          2             8.14   
          3             8.18   
          Studio       10.13   
Larnaca   1             8.70   
          2             4.93   
          3             7.82   
          4             7.09   
          Studio        8.19   
Limassol  1             9.31   
          2             5.10   
          3            28.63   
          4             4.24   
          5             7.16   
          6 and more    4.08   
          Studio        2.07   
Nicosia   1             4.42   
          2             6.02   
          3          4209.52   
          4             6.

houses


/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/bazaraki/.venv/lib/python3.10/site-packages/pandas/core/nanops.py:1016: RuntimeWarning:

invalid value encountered in subtract

/home/ubuntu/

result  \
                        count   
city      Bedrooms              
Famagusta 2            469.57   
          3            440.82   
          4            288.89   
          5            100.00   
Larnaca   1            960.00   
          2           1118.64   
          3            666.06   
          4            432.49   
          5            278.05   
          6             92.31   
          7            266.67   
Limassol  1           5333.33   
          2           1124.21   
          3           1320.34   
          4            671.64   
          5            519.84   
          6            225.88   
          7            387.10   
          8            150.00   
Nicosia   1           2400.00   
          2           1160.00   
          3            787.95   
          4            465.75   
          5            300.00   
          6            313.04   
          7            240.00   
          8           2400.00   
Paphos    1           1200.00   
          2            872.73   
          3            718.15   
          4            732.20   
          5            566.04   
          6            635.29   
          7            600.00   
Limassol  10 and more     NaN   
          Studio          NaN   
Nicosia   Studio          NaN   
Paphos    Studio          NaN   
Famagusta 6               NaN   
Larnaca   10 and more     NaN   
          8               NaN   
          9               NaN   
Limassol  9               NaN   
Nicosia   10 and more     NaN   
Paphos    8               NaN   

                             \
                       mean   
city      Bedrooms            
Famagusta 2             inf   
          3            4.04   
          4            3.53   
          5            4.89   
Larnaca   1            5.63   
          2             inf   
          3            0.00   
          4             inf   
          5            0.00   
          6            7.73   
          7            3.77   
Limassol  1            0.00   
          2            8.40   
          3             NaN   
          4            0.00   
          5            0.00   
          6            6.09   
          7            5.36   
          8            2.75   
Nicosia   1             inf   
          2            5.89   
          3            5.41   
          4             inf   
          5            4.89   
          6            6.32   
          7           11.61   
          8            5.02   
Paphos    1            2.96   
          2           11.17   
          3            6.29   
          4            3.73   
          5            4.56   
          6             inf   
          7            6.92   
Limassol  10 and more   NaN   
          Studio        NaN   
Nicosia   Studio        NaN   
Paphos    Studio        NaN   
Famagusta 6             NaN   
Larnaca   10 and more   NaN   
          8             NaN   
          9             NaN   
Limassol  9             NaN   
Nicosia   10 and more   NaN   
Paphos    8             NaN   

                              \
                         std   
city      Bedrooms             
Famagusta 2              NaN   
          3             2.77   
          4             2.47   
          5             0.85   
Larnaca   1            10.99   
          2              NaN   
          3              NaN   
          4              NaN   
          5              NaN   
          6             4.52   
          7             2.46   
Limassol  1              NaN   
          2            48.19   
          3              NaN   
          4              NaN   
          5              NaN   
          6             8.09   
          7             6.03   
          8              NaN   
Nicosia   1              NaN   
          2             3.93   
          3             5.27   
          4              NaN   
          5             3.63   
          6             4.68   
          7            18.09   
          8          

In [24]:
df.query("Bedrooms == '2' and city=='Limassol'").price_per_sqm.hist(bins=100)

In [ ]:
px.histogram(df.query("Bedrooms == '1' and price_per_sqm < 200"), x="price_per_sqm", nbins=100, color="city", barmode="overlay").show()


In [19]:
px.histogram(df, x="price", barmode='overlay', facet_col="city", color="Bedrooms").show()


In [47]:
df.sort_values("views", ascending=False)


,url,title,price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,description,images,Property area,Pets,Type,Floor,Parking,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,Postal code,Construction year,Reference number,city,district
1018,https://www.bazaraki.com/adv/4025924_1-bedroom-apartment-to-rent/,Studio apartment to rent,600,17.14,"Limassol, Trimiklini",18.11.2024 13:34,4025924,NaN,27225,34.85,32.91,Brand new large studios for rent.\nThe studios are fully furnished. In the price is included ele...,"[https://cdn1.bazaraki.com/media/cache1/82/90/829044a36d68bee9ea901a0c162dd458.webp, https://cdn...",35 m²,Not allowed,Apartment,NaN,Covered,Fully Furnished,"Garden, Balcony",No,"Full, all rooms",N/A,Studio,1,4730.00,2023,NaN,Limassol,Trimiklini
4211,https://www.bazaraki.com/adv/4852027_studio-apartment-to-rent/,Studio apartment to rent,450,inf,"Larnaca, Perivolia Larnakas",13:30,4852027,NaN,14326,NaN,NaN,𝐍𝐄𝐖 𝐒𝐓𝐔𝐃𝐈𝐎 𝐀𝐏𝐀𝐑𝐓𝐌𝐄𝐍𝐓𝐒 𝐅𝐎𝐑 𝐑𝐄𝐍𝐓\nFully Furnished\nAircondition\nSmart TVs\nWi-Fi internet\nWashin...,"[https://cdn1.bazaraki.com/media/cache1/bb/e8/bbe8dae73c9864bc080fead856397954.webp, https://cdn...",0 m²,Not allowed,Apartment,1st,Covered,Fully Furnished,NaN,Yes,"Full, all rooms",A,Studio,1,7560.00,2023,NaN,Larnaca,Perivolia Larnakas
425,https://www.bazaraki.com/adv/4627575_2-bedroom-apartment-to-rent/,1-bedroom apartment to rent,800,16.00,"Limassol, Germasogeia Tourist Area",30.11.2024 22:24,4627575,NaN,5291,NaN,NaN,Experience Limassol from the best area close to the beach . Calls after 10.00. only serious per...,"[https://cdn1.bazaraki.com/media/cache1/50/7d/507dc16d8a2b10759bb0034d7724adef.webp, https://cdn...",50 m²,Not allowed,Apartment,2nd,No,Fully Furnished,"Elevator, Alarm, Balcony",No,Partly,C,1,1,4040.00,Older,NaN,Limassol,Germasogeia Tourist Area
3929,https://www.bazaraki.com/adv/5256064_studio-apartment-to-rent/,Studio apartment to rent,300,4.62,"Nicosia, Klirou",03.12.2024 12:11,5256064,NaN,5251,NaN,NaN,"Studio for rent in Klirou, fully furnished, preferred people from Europe who work in Cyprus.\nΕν...","[https://cdn1.bazaraki.com/media/cache1/aa/d8/aad85bf8dccca73093c28cc66a1bd898.webp, https://cdn...",65 m²,Not allowed,Apartment,Ground floor,NaN,Fully Furnished,NaN,NaN,"Full, all rooms",B,Studio,1,NaN,NaN,NaN,Nicosia,Klirou
2938,https://www.bazaraki.com/adv/4776408_3-bedroom-apartment-to-rent/,2-bedroom apartment to rent,700,7.61,"Larnaca, Livadia Larnakas",19.11.2024 06:41,4776408,NaN,4999,NaN,NaN,Ενοικιαζεται διαμερισμα στη Λαρνακα. Ενοικιαζεται ΜΟΝΟ για γυναικες ή ενα ζευγαρι.Διαθετει 2 δωμ...,"[https://cdn1.bazaraki.com/media/cache1/f9/27/f9272266c5e7a12b27bd6fd3555cd05b.webp, https://cdn...",92 m²,Not allowed,Apartment,2nd,Uncovered,Fully Furnished,Balcony,No,"Full, all rooms",A,2,2,6052.00,2011,NaN,Larnaca,Livadia Larnakas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4288,https://www.bazaraki.com/adv/5548602_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,2300,23.47,"Limassol, Limassol - Tsirion",09:13,5548602,NaN,1,34.70,33.02,"New apartment, built in 2023. 2 bedrooms. Fully furnished.","[https://cdn1.bazaraki.com/media/cache1/af/f5/aff5abdbc5d43cae36407a5e7f45780c.webp, https://cdn...",98 m²,Not allowed,Apartment,2nd,Covered,Fully Furnished,"Elevator, Balcony",No,"Full, all rooms",A,2,1,3081.00,2023,NaN,Limassol,Limassol - Tsirion
4228,https://www.bazaraki.com/adv/5519958_3-bedroom-apartment-to-rent/,3-bedroom apartment to rent,3000,16.67,"Nicosia, Egkomi",14:40,5519958,NaN,1,35.16,33.33,Rent this beautiful Flat through www.rentonline.today and save 30% on your first monthly rent!. ...,"[https://cdn1.bazaraki.com/media/cache1/72/bb/72bb10111004854bce49ff4443d641a1.webp, https://cdn...",180 m²,Not allowed,Apartment,Ground floor,Covered,Fully Furnished,NaN,NaN,"Full, all rooms",A,3,3,NaN,2023,rentonline-94,Nicosia,Egkomi
4229,https://www.bazaraki.com/adv/54755

In [ ]:
saledf = pd.read_json("output/_real-estate-for-sale_apartments-flats__20241207_133456.jsonl", lines=True)
saledf

,url,title,price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,description,images,Property area,Type,Floor,Parking,Condition,Furnishing,Included,Air conditioning,Energy Efficiency,Bedrooms,Square meter price,Bathrooms,Reference number,Postal code,Construction year,Online viewing,Registration block,Registration number
0,https://www.bazaraki.com/adv/5270074_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,110000,2558.0,"Famagusta, Kapparis",Yesterday 16:43,5270074,NaN,1313,NaN,NaN,"A 1-bedroom Second Floor Apartment in Kapparis, Famagusta. The Apartment is located at the top f...","[https://cdn1.bazaraki.com/media/cache1/16/41/1641c407b2fd66410b3a69c9c71fcfc4.webp, https://cdn...",43 m²,Apartment,2nd,Covered,Resale,Fully Furnished,"Pool, Balcony, Storage room","Full, all rooms",N/A,1,€2.558 /m²,1,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.bazaraki.com/adv/5513457_3-bedroom-penthouse-for-sale/,3-bedroom penthouse fоr sаle,550000,4583.0,"Limassol, Agios Athanasios",03.12.2024 23:05,5513457,NaN,5,NaN,NaN,"Reference ID: #SA34867Price details: €550,000 +VATA new project for a three-floor residential bu...","[https://cdn1.bazaraki.com/media/cache1/e7/b0/e7b0e9e3b0ffc1ead147229f44bafbcd.webp, https://cdn...",120 m²,Penthouse,3rd,Covered,Brand new,Unfurnished,"Storage room, Balcony",No,N/A,3,€4.583 /m²,2,SA34867,NaN,NaN,NaN,NaN,NaN
2,https://www.bazaraki.com/adv/5225611_3-bedroom-apartment-for-sale/,3-bedroom apartment fоr sаle,600000,5085.0,"Limassol, Limassol - Neapolis",Yesterday 16:42,5225611,NaN,23,34.684042,33.031313,"Luxury Sea view apartment, with panoramic sea views, within walking distance to the beach and th...","[https://cdn1.bazaraki.com/media/cache1/84/2f/842f66392ea01b68c17398a3b9967564.webp, https://cdn...",118 m²,Apartment,NaN,No,Resale,NaN,NaN,NaN,B+,3,€5.085 /m²,4,7444,3030.0,2023,NaN,NaN,NaN
3,https://www.bazaraki.com/adv/5294642_studio-apartment-for-sale/,Studio apartment fоr sаle,197000,6567.0,"Limassol, Historical Center",04.12.2024 08:43,5294642,NaN,7,34.679846,33.044669,This project features 25 serviced studio apartments located in Limassol's Old Town.\nAmenities i...,"[https://cdn1.bazaraki.com/media/cache1/0f/d6/0fd68165844de93bfca2214d0604c3e5.webp, https://cdn...",30 m²,Apartment,3rd,No,Brand new,NaN,Elevator,"Full, all rooms",A,Studio,€6.567 /m²,NaN,52865,NaN,NaN,No,NaN,NaN
4,https://www.bazaraki.com/adv/5556626_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,169500,1863.0,"Paphos, Geroskipou",03.12.2024 15:01,5556626,NaN,37,34.761539,32.458060,This spacious first floor fully furnished apartment is in a popular complex overlooking a large ...,"[https://cdn1.bazaraki.com/media/cache1/a3/bc/a3bc868e178410266c6a166a70084d4b.webp, https://cdn...",91 m²,Apartment,NaN,NaN,Resale,NaN,NaN,NaN,N/A,2,€1.863 /m²,1,MDVGPH2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12666,https://www.bazaraki.com/adv/5513325_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,252000,4000.0,"Paphos, Tombs Of the Kings",27.11.2024 19:35,5513325,NaN,68,34.774553,32.404895,"YL Luxury Homes Ltd presents a new luxury and modern development in the heart of Kato Pafos, clo...","[https://cdn1.bazaraki.com/media/cache1/a5/f2/a5f2df0c92055456183edc04b3265cbd.webp, https://cdn...",63 m²,Apartment,1st,Covered,Under construction,NaN,"Pool, Elevator, Balcony, Storage room",NaN,A,1,€4.000 /m²,1,NaN,NaN,NaN,NaN,NaN,NaN
12667,https://www.bazaraki.com/adv/5493768_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,215000,2312.0,"Nicosia, Stelmek Area",28.11.2024 10:00,5493768,NaN,363,35.140516,33.300250,"Modern Living in Stelmek, Nicosia – Limited Apartments Available!\nBe part of this exciting new ...","[https://cdn1.bazaraki.com/media/cache1/85/de/85de8c5d2727aef312fc7f49a7b028ca.webp, https://cdn...",93 m²,Apartment,NaN,Covered,Brand new,NaN,"Garden, Elevator, Balcony, Storage room",NaN,A,2,€2.312 /

In [ ]:
saledf.iloc[0].T

url                                                     https://www.bazaraki.com/adv/5270074_1-bedroom-apartment-for-sale/
title                                                                                         1-bedroom apartment fоr sаle
price                                                                                                               110000
price_per_sqm                                                                                                       2558.0
location                                                                                               Famagusta, Kapparis
posted                                                                                                     Yesterday 16:43
ad_id                                                                                                              5270074
reference_number                                                                                                       NaN
views           

In [ ]:
saledf[["city", "district"]] = saledf.location.str.split(",", expand=True)